# Многопроцессность и многопоточность

### В чем разница между concurrency и parallelism?

Конкурентность — это выполнение задач за определённое время (например, есть 5 процессов и все они в сумме выполняются в течение 60 минут по очереди). Важная деталь заключается в том, что задачи необязательно выполняются одновременно, поэтому их можно разделить на более мелкие и чередующиеся.

<center>
<img src="https://iamluminousmen-media.s3.amazonaws.com/media/concurrency-and-parallelism-are-different/concurrency-and-parallelism-are-different-2.jpg" width=480>
</center>

Параллелизм — это выполнение задач в одно и то же время (например, есть 5 задач, каждая из них выполняется в течение 60 минут). Само название подразумевает, что они выполняются параллельно.

<center>
<img src="https://iamluminousmen-media.s3.amazonaws.com/media/concurrency-and-parallelism-are-different/concurrency-and-parallelism-are-different-3.jpg" width=480>
</center>

В Python есть подержка обоих механизмов:
- `asyncio` – конкурентность;
- `multiprocessing` – параллелизм.

### В чем разница между процессами и потоками?

Процесс — запущенная программа. У каждого процесса есть изолированное от других процессов состояние:
- виртуальное адресное пространство;
- указатель на исполняемую инструкцию;
- стек вызовов;
- системные ресурсы, например, открытые файловые дескрипторы.

Процессы удобны для одновременного выполнения нескольких задач.

Поток похож на процесс тем, что его исполнение происходит независимо от других потоков (и процессов). В отличие от процесса поток исполняется внутри процесса и разделяет с ним адресное пространство и системные ресурсы.

Потоки удобны для одновременного выполнения нескольких задач, которым требуется доступ к разделяемому состоянию.

### Все ли так хорошо с потоками в Python?

Если два или более потока попытаются манипулировать одним и тем же объектом в одно и то же время, то неизбежно возникнут проблемы. Глобальная блокировка интерпретатора (GIL) исправляет это. В любой момент времени действия может выполнять только один поток. Python автоматически переключается между потоками, когда в этом возникает необходимость.

GIL (global interpreter lock) — это мьютекс, который гарантирует, что в каждый момент времени только один поток имеет доступ к внутреннему состоянию интерпретатора.

Почему GIL для Python так важен? Дело в том, что у каждого объекта есть специальный счетчик ссылок на этот объект. Объект удаляется GC (garbage collector), когда число ссылок на объект равно 0.

In [ ]:
import sys

In [ ]:
a = []
b = a

sys.getrefcount(a)  # почему ответ такой?

In [ ]:
del a
sys.getrefcount(b)

Продемонстрируем, что GIL действительно не ускоряет python код.

In [ ]:
import random

def countdown(n):
    while n > 0:
        a = random.randint(-3, 3)
        b = random.randint(-3, 3)
        c = random.randint(-3, 3)
        
        d = (a - b) ** 2 + (a - c) ** 2 + (b - c) ** 2
        n -= 1

In [ ]:
%%time

countdown(100_000)

Для работы с потоками в python есть модуль `threading`.

In [ ]:
%%time

from threading import Thread

# создаем потоки
t1 = Thread(target=countdown, args=(50_000, ))
t2 = Thread(target=countdown, args=(50_000, ))

# запускаем потоки
t1.start()
t2.start()

# дожидаемся потоки
t1.join()
t2.join()

К счастью, не все так плохо – некоторые операции позволяют "отпустить" GIL:
- работа с файловыми дескрипторами (I/O операции);
- вызов C/C++ кода.

In [ ]:
import requests

In [ ]:
%%time

pages = []

for post_id in range(349000, 349200, 10):
    post_url = f'https://habr.com/post/{post_id}/'
    page = requests.get(post_url)
    pages.append(page)
    
len(pages)

В `python` есть специальные удобные пулы процессов/потоков, которые очень удобно использовать в простых задачах, наподобие предыдущей.

In [ ]:
from multiprocessing.dummy import Pool as ThreadPool


def retrieve(post_id):
    post_url = f'https://habr.com/post/{post_id}/'
    page = requests.get(post_url)
    return page

In [ ]:
%%time

with ThreadPool(2) as pool:
    pages = pool.map(retrieve, range(349000, 349200, 10))
    pool.close()
    pool.join()
    
len(pages)

При работе с потоками важно помнить про синхронизацию.

In [ ]:
def increment():
    global x
    x += 1

def thread_task():
    for _ in range(100000):
        increment()

In [ ]:
x = 0

t1 = Thread(target=thread_task)
t2 = Thread(target=thread_task)
  
t1.start()
t2.start()
  
t1.join()
t2.join()

x

Добавим специальный объект (мьютекс) для синхронизации потоков.

In [ ]:
from threading import Lock


def thread_task(lock):
    for _ in range(100000):
        with lock:
            increment()
            
        # same as:
        # lock.acquire()
        # increment()
        # lock.release()

In [ ]:
x = 0

lock = Lock()

t1 = Thread(target=thread_task, args=(lock, ))
t2 = Thread(target=thread_task, args=(lock, ))
  
t1.start()
t2.start()
  
t1.join()
t2.join()

x

### Процессы

Процессы можно также создавать по-одиночке, как и потоки.

In [ ]:
%%time

countdown(100_000)

In [ ]:
%%time

from multiprocessing import Process

# создаем процессы
p1 = Process(target=countdown, args=(50_000, ))
p2 = Process(target=countdown, args=(50_000, ))

# запускаем процессы
p1.start()
p2.start()

# дожидаемся процессы
p1.join()
p2.join()

In [ ]:
p = Process(target=countdown, args=(100_000, ))
p.start()

In [ ]:
p.name, p.pid

In [ ]:
%%time

p.join()
p.exitcode

Или группами...

In [ ]:
from multiprocessing import Pool

In [ ]:
%%time

with Pool(2) as pool:
    pages = pool.map(retrieve, range(349000, 349200, 10))
    pool.close()
    pool.join()
    
len(pages)

Пример работы с разделяемым объектом

In [ ]:
from multiprocessing import Value, Lock

In [ ]:
def increment():
    global x
    x.value += 1

def process_task():
    for _ in range(100000):
        increment()

In [ ]:
x = Value('i', 0)

p1 = Process(target=process_task)
p2 = Process(target=process_task)
  
p1.start()
p2.start()
  
p1.join()
p2.join()

x.value

In [ ]:
def increment():
    global x
    x.value += 1

def process_task(lock):
    for _ in range(100000):
        with lock:
            increment()

In [ ]:
x = Value('i', 0)

lock = Lock()

p1 = Process(target=process_task, args=(lock, ))
p2 = Process(target=process_task, args=(lock, ))
  
p1.start()
p2.start()

p1.join()
p2.join()

x.value